In [1]:
import logging

logging.basicConfig(level=logging.DEBUG)

### Load data

In [1]:
import sys
from pathlib import Path

# Add project root to Python path
project_root = str(Path.cwd().parent)  # Assumes notebook is in integration/ directory
if project_root not in sys.path:
    sys.path.append(project_root)

import datetime
from tests.integration.helpers import get_data_for_ticker

prices = get_data_for_ticker("SHIBUSDT", start_date=datetime.datetime(2024, 1, 1), end_date=datetime.datetime(2024, 11, 25))

### Calculate Zigzag

In [2]:
from auto_chart_patterns.zigzag import Zigzag

zigzag = Zigzag(backcandles=2, forwardcandles=2, pivot_limit=100, offset=0)
zigzag.calculate(prices)

for pivot in zigzag.zigzag_pivots:
    print(f"Pivot: time={pivot.point.time}, index={pivot.point.index}, price={pivot.point.price}, direction={pivot.direction}")


Pivot: time=2024-10-31 00:00:00, index=280, price=1.771e-05, direction=-1
Pivot: time=2024-10-29 00:00:00, index=278, price=1.983e-05, direction=2
Pivot: time=2024-10-25 00:00:00, index=274, price=1.583e-05, direction=-2
Pivot: time=2024-10-19 00:00:00, index=268, price=1.955e-05, direction=2
Pivot: time=2024-10-10 00:00:00, index=259, price=1.605e-05, direction=-1
Pivot: time=2024-10-06 00:00:00, index=255, price=1.885e-05, direction=1
Pivot: time=2024-10-03 00:00:00, index=252, price=1.548e-05, direction=-1
Pivot: time=2024-09-27 00:00:00, index=246, price=2.169e-05, direction=2
Pivot: time=2024-09-18 00:00:00, index=237, price=1.296e-05, direction=-1
Pivot: time=2024-09-13 00:00:00, index=232, price=1.411e-05, direction=2
Pivot: time=2024-09-11 00:00:00, index=230, price=1.293e-05, direction=-1
Pivot: time=2024-09-09 00:00:00, index=228, price=1.376e-05, direction=1
Pivot: time=2024-09-06 00:00:00, index=225, price=1.231e-05, direction=-2
Pivot: time=2024-08-24 00:00:00, index=212, 

### Find Reversal Patterns

In [3]:
from auto_chart_patterns.reversal_patterns import find_reversal_patterns, ReversalPatternProperties

scan_props = ReversalPatternProperties(min_periods_lapsed=8, flat_ratio=0.1)
patterns = []
for i in range(0, len(zigzag.zigzag_pivots)):
    find_reversal_patterns(zigzag, i, scan_props, patterns)

for pattern in patterns:
    print(f"{pattern.pattern_name}: {pattern.pivots[0].point.index} - {pattern.pivots[-1].point.index}")

Double Bottoms: 246 - 268
Triple Bottoms: 212 - 246
Double Bottoms: 156 - 182
Double Tops: 121 - 140
Double Bottoms: 45 - 64
Double Tops: 35 - 51
Double Bottoms: 29 - 45


### Find RSI Divergences

In [4]:
from auto_chart_patterns.rsi_div_patterns import find_rsi_divergences, RsiDivergenceProperties

scan_props = RsiDivergenceProperties(min_periods_lapsed=5, min_change_pct=0.005)
patterns = []
find_rsi_divergences(backcandles=2, forwardcandles=2, properties=scan_props, patterns=patterns, df=prices)

for pattern in patterns:
    print(f"{pattern.pattern_name}: {pattern.pivots[0].point.index} - {pattern.pivots[-1].point.index}")

Hidden Bearish: 98 - 112
Hidden Bearish: 173 - 178
Hidden Bearish: 178 - 183
Bearish: 269 - 278
Bullish: 128 - 132
Hidden Bullish: 146 - 154
Bullish: 185 - 195
Bullish: 199 - 204
Bullish: 220 - 225
Hidden Bullish: 230 - 235
Hidden Bullish: 242 - 252
